In [ ]:
from spatial_peaks import *

#create two filters: laplacian and gaussian
laplacian = np.zeros([3,3])
laplacian[np.unravel_index(range(9)[1::2], (3,3))] = -1
laplacian[1,1] = 4

gaussian = np.ones([3,3])
gaussian[np.unravel_index(range(9)[1::2], (3,3))] = 2
gaussian[1,1] = 4
gaussian *= 1/16

prediction1 = []
for ind, image in enumerate(images):
    # image_curvature = convolve(image, laplacian) # not used
    smooth_image = convolve(image, gaussian) # smoothen it
    smooth_curvature = convolve(smooth_image, laplacian) # and then find the curvature of the smoothed image
    bulged_mask = smooth_curvature>smooth_curvature.max() * 0.05 # find where does the bulge mask
    local_max_mask = is_local_max_2d_wide(image)

    pred1 = np.logical_and(local_max_mask, bulged_mask).sum() # >90 % accuracy
    """
    Failed ideas:
    pred1 = (image[is_local_max_2d_wide(image_curvature, diagonal=True)] > np.percentile(image.flatten(), 245/256*100)).sum() # 87.76 % accuracy
    pred2 = (image[is_local_max_2d_wide(smooth_image, diagonal=True)] > np.percentile(image.flatten(), 245/256*100)).sum() # 85.13 % accuracy
    pred2 = (image[peak_local_max(image_curvature)]>np.percentile(image.flatten(), 250/256*100)).sum()
    pred3 = (image[peak_local_max(smooth_image)]>np.percentile(image.flatten(), 250/256*100)).sum()
    """
    prediction1.append(pred1)

# get the labels (truths)
labels = 2 - (energies==0).sum(axis=1)
# Peak must be at the center of a blob (contiguous area with 3+ pixels) where curvature is negative in both directions

truth_pred = ary([labels, prediction1]).T

In [ ]:
print(confusion_matrix(labels, prediction1)) # display result
print("The columns are the predicted number of events found in that image (ascending from 1);")
print("the rows are the true number of events actually existing in that image (ascending from 1, top to bottom).")

In [ ]:
# <CHANGE HERE>CHOOESE TYPE OF ERROR TO BE EXAMINED HERE
predicted_as = 1 # event(s)
actually_has = 2 # event(s)
# END OF USER DEFINED SECTION </CHANGE HERE>

chosen_to_be_examined = (truth_pred==[actually_has,predicted_as]).all(axis=1)
for num, image in enumerate(images[chosen_to_be_examined][:num_samples]):
    ind = np.where(chosen_to_be_examined)[0][num]
    replay = True
    while replay:
        # introduce information about neighbourhood
        image_curvature = convolve(image, laplacian) # introduce some global information
        smooth_image = convolve(image, gaussian)
        smooth_curvature = convolve(smooth_image, laplacian)
        bulged_mask = smooth_curvature>smooth_curvature.max()*0.05
        local_max_mask = is_local_max_2d_wide(image)

        for img, name in zip([image, image_curvature, smooth_image, smooth_curvature, bulged_mask, local_max_mask],
                            ['image', 'image_curvature', 'smooth_image', 'smooth_curvature', 'bulged_mask', 'local_max_mask']):
            sns.heatmap(img)
            plt.title(f'{ind}: '+name+f' #events={labels[ind]}')
            plt.show()
        if len(input('Enter any character before to move onto the next plot, or ctrl+c to stop. Press only enter to replay this plot.'))>0:
            replay=False